In [ ]:
# for id_ in G.nodes:
#     if '...","' in id_:
#         print(G.nodes[id_])

In [ ]:
try:
    %load_ext lab_black
except ModuleNotFoundError:
    pass

In [ ]:
import os
import sys
import random
import functools
import shelve
import hashlib
import logging

import networkx as nx
import numpy as np
import webbrowser
from time import time
from IPython.core.display import display, HTML
from IPython.display import Javascript
from scipy.cluster.hierarchy import cut_tree, to_tree, leaves_list
from ipywidgets import Button, HBox, VBox, Output, Layout, Image, Checkbox
from ipyevents import Event
import matplotlib.pyplot as plt
from krakow import krakow
from krakow.utils import (
    normalized_dasgupta_cost,
    split_into_n_children,
)

sys.path.append(os.path.abspath(".."))


plt.style.use("dark_background")

from yourtube.scraping import scrape_from_list
from yourtube.file_operations import (
    save_graph,
    load_graph,
    id_to_url,
    cluster_cache_path,
)

# logging.basicConfig(level=logging.INFO)

In [ ]:
id_to_thumbnail = "https://i.ytimg.com/vi/{}/mqdefault.jpg"
# id_to_thumbnail = "https://i.ytimg.com/vi/{}/maxresdefault.jpg"
# hq and sd usually has black stripes
# mq < hq < sd < maxres

In [ ]:
G = load_graph()

In [ ]:
# filtering functions


def added_in_last_n_years(G, ids, n=5):
    seconds_in_month = 60 * 60 * 24 * 30.4
    seconds_in_year = seconds_in_month * 12
    start_time = time() - seconds_in_year * n
    # round start_time to months, to prevent clustering being recalculated too frequently
    # returned ids will change only each month, so the cached value will be used
    start_time = start_time // seconds_in_month * seconds_in_month

    filterd_ids = []
    for id_ in ids:
        node = G.nodes[id_]
        if "time_added" not in node:
            continue
        if start_time < node["time_added"]:
            filterd_ids.append(id_)
    return filterd_ids


def only_not_watched(G, ids):
    filterd_ids = []
    for id_ in ids:
        node = G.nodes[id_]
        if not node.get("watched_times"):
            filterd_ids.append(id_)
    return filterd_ids


def only_watched(G, ids):
    filterd_ids = []
    for id_ in ids:
        node = G.nodes[id_]
        if node.get("watched_times"):
            filterd_ids.append(id_)
    return filterd_ids


def get_neighborhood(G, ids):
    out_edges = G.out_edges(ids)
    return list(G.edge_subgraph(out_edges).nodes)


def from_category(G, ids, categories):
    # note: if some of the ids hasn't beed scraped, they will be filtered out
    # regardless of their category (because it isn't known)
    filterd_ids = []
    for id_ in ids:
        node = G.nodes[id_]
        if node.get("category") in categories:
            filterd_ids.append(id_)
    return filterd_ids


def not_down(G, ids):
    filterd_ids = []
    for id_ in ids:
        node = G.nodes[id_]
        if not node.get("is_down"):
            filterd_ids.append(id_)
    return filterd_ids

In [ ]:
def cluster_subgraph(nodes_to_cluster, G, balance=2):
    # use cache
    sorted_nodes = sorted(nodes_to_cluster)
    unique_node_string = "".join(sorted_nodes)
    unique_node_string += str(balance)
    node_hash = hashlib.md5(unique_node_string.encode()).hexdigest()
    with shelve.open(cluster_cache_path) as cache:
        if node_hash in cache:
            return cache[node_hash]

    RecentDirected = G.subgraph(nodes_to_cluster)
    Recent = RecentDirected.to_undirected()

    # choose only the biggest connected component
    components = sorted(nx.connected_components(Recent), key=len, reverse=True)
    # for el in components[:5]:
    #     print(len(el))
    main_component = components[0]
    Main = Recent.subgraph(main_component)

    D = krakow(Main, balance=balance)
    tree = to_tree(D)

    img = plot_dendrogram(D, clusters_limit=100, width=17.8, height=1.5)
    # normalized_dasgupta_cost(Main, D)

    def convert_leaf_values_to_original_ids(tree, Graph):
        main_ids_list = np.array(Graph.nodes)

        def substitute_video_id(leaf):
            leaf.id = main_ids_list[leaf.id]

        tree.pre_order(substitute_video_id)

    convert_leaf_values_to_original_ids(tree, Main)

    # save to cache
    with shelve.open(cluster_cache_path) as cache:
        cache[node_hash] = tree, img
    return tree, img

In [ ]:
# B = load_graph("basia")

In [ ]:
def select_nodes_to_cluster(G, use_watched=False, categories="all"):
    sources = added_in_last_n_years(G, G.nodes, n=5)
    sources = not_down(G, sources)

    if use_watched:
        watched = only_watched(G, G.nodes)
        sources = set(sources) | set(watched)

    if categories != "all":
        # the sources are scraped, so each has some category, so filtering is lossless
        sources = from_category(G, sources, categories)

    return get_neighborhood(G, sources)

In [ ]:
# ranking functions


def liked_to_views_ratio(G, id_):
    node = G.nodes[id_]
    try:
        return node["like_count"] / node["view_count"]
    except (KeyError, TypeError, ZeroDivisionError):
        return -1

In [ ]:
class Categories:
    def __init__(self, G):
        # get all categories
        categories = []
        for id_, node in G.nodes.data():
            if "category" in node:
                categories.append(node["category"])

        # sort categories by their importance
        counts = Counter(categories)
        counts = list(counts.items())
        counts.sort(key=lambda i: i[1], reverse=True)
        # print(counts)
        category_names = [name for name, count in counts]

        # construct category checkboxes
        self.category_checkboxes = []
        for category_name in category_names:
            checkbox = Checkbox(description=category_name, value=True)
            self.category_checkboxes.append(checkbox)

        select_all_button = Button(description="Select all")
        deselect_all_button = Button(description="Deselect all")
        select_all_button.on_click(self.select_all)
        deselect_all_button.on_click(self.deselect_all)

        self.checkboxes_vbox = VBox(
            [select_all_button, deselect_all_button] + self.category_checkboxes
        )

    def select_all(self, _):
        for category_checkbox in self.category_checkboxes:
            category_checkbox.value = True

    def deselect_all(self, _):
        for category_checkbox in self.category_checkboxes:
            category_checkbox.value = False

    def get_checked_categories_list(self):
        checked = []
        for category_checkbox in self.category_checkboxes:
            if category_checkbox.value:
                checked.append(category_checkbox.description)
        return checked


class TreeClimber:
    def __init__(self, num_of_groups, videos_in_group, message_output, display_updater):
        self.num_of_groups = num_of_groups
        self.videos_in_group = videos_in_group
        self.message_output = message_output
        self.display_updater = display_updater

    def reset(self, tree):
        self.tree = tree
        self.path = []
        self.children, self.grandchildren = self.new_offspring(self.tree)
        self.display_updater(None)

    def choose_column(self, _, i):
        self.message_output.clear_output()

        new_tree = self.children[i]
        try:
            new_children, new_grandchildren = self.new_offspring(new_tree)
        except ValueError:
            with self.message_output:
                print("already on the lowest cluster")
            return

        self.path.append(self.tree)
        self.tree = new_tree
        self.children = new_children
        self.grandchildren = new_grandchildren
        self.display_updater(None)

    def go_back(self, _):
        self.message_output.clear_output()
        if self.path == []:
            with self.message_output:
                print("already on the highest cluster")
            return
        self.tree = self.path.pop()
        self.children, self.grandchildren = self.new_offspring(self.tree)
        self.display_updater(None)

    def new_offspring(self, new_tree):
        new_children = split_into_n_children(new_tree, n=self.num_of_groups)
        new_grandchildren = [
            split_into_n_children(new_child, n=self.videos_in_group)
            for new_child in new_children
        ]
        return new_children, new_grandchildren


# TODO move this to krakow
from scipy.cluster.hierarchy import dendrogram
import io


def plot_dendrogram(D, clusters_limit=100, width=10, height=4):
    # a hack to disable plotting, only return the image
    was_interactive = plt.isinteractive()
    plt.ioff()

    fig = plt.figure(figsize=(width, height))
    # display logarithm of cluster distances
    Dlog = D.copy()
    Dlog[:, 2] = np.log(Dlog[:, 2])
    # cut off the bottom part of the plot as it's not informative
    Dlog[:, 2][:-clusters_limit] *= 0
    Dlog[-clusters_limit:, 2] = Dlog[-clusters_limit:, 2] - Dlog[-clusters_limit, 2]

    dendrogram(Dlog, leaf_rotation=90.0, truncate_mode="lastp", p=clusters_limit)
    plt.axis("off")
    img = io.BytesIO()
    plt.savefig(img, bbox_inches="tight")

    # revert to the previous plt state
    if was_interactive:
        plt.ion()
    return img

In [ ]:
class Recommender:
    def __init__(self, G, cutoff=0.7, seed=None):
        assert 0 < cutoff < 1
        self.cutoff = cutoff
        self.seed = seed if seed is not None else random.randint(1, 1000000)

    def compute_node_ranks(self, ids):
        """This function must be called on given ids before we can use recommender on those ids."""

        source_videos = added_in_last_n_years(G, ids)
        # note: these may not really be source videos!

        # compute node ranks
        self.node_ranks = dict()
        source_videos_set = set(source_videos)
        for id_ in ids:
            in_edges = G.in_edges(id_)
            in_nodes = {u for u, v in in_edges}
            rank = len(in_nodes & source_videos_set)
            self.node_ranks[id_] = rank

    def get_index(self, length):
        np.random.seed(self.seed + length)
        position = np.random.triangular(self.cutoff, 1, 1)
        # other potential distributions are: exponential, lognormal
        index = int(length * position)
        # just to be sure, that we don't get IndexError due to numerical rounding
        index = np.clip(index, 0, length - 1)
        return index

    def recommend_by_in_degree(self, ids):
        # if there if nothing, return nothing
        if len(ids) == 0:
            return None

        index = self.get_index(len(ids))

        ranks = [self.node_ranks[id_] for id_ in ids]
        # find the index on ids list of the video with index'th smallest rank
        index_on_ids_list = np.argpartition(ranks, index)[index]
        chosen_id = ids[index_on_ids_list]
        return chosen_id

In [ ]:
class UI:
    def __init__(
        self,
        G,
        num_of_groups,
        videos_in_group,
        clustering_balance,
        recommendation_cutoff,
        column_width,
        seed,
        filter_categories=False,
    ):
        # filter_categories is temporarily broken

        self.G = G
        self.num_of_groups = int(num_of_groups)
        self.videos_in_group = int(videos_in_group)
        # to have less possible balance values, to better use cache
        self.clustering_balance = round(clustering_balance, 1)
        self.filter_categories = filter_categories
        self.column_width = column_width
        self.grid_gap = 20

        self.recommender = Recommender(G, recommendation_cutoff, seed)

        self.image_output = Output()
        self.message_output = Output()
        self.video_wall = Output()
        self.tree_climber = TreeClimber(
            self.num_of_groups,
            self.videos_in_group,
            self.message_output,
            self.update_displayed_videos,
        )

        go_back_button = Button(description="Go back")
        go_back_button.on_click(self.tree_climber.go_back)
        self.hide_watched_checkbox = Checkbox(description="Hide watched", value=False)
        self.hide_watched_checkbox.observe(self.update_displayed_videos, names="value")
        self.use_watched_checkbox = Checkbox(
            description="(advanced) Use watched videos", value=False
        )
        if len(added_in_last_n_years(G, G.nodes)) < 400:
            # if there are too few videos in playlists, it's better to also use watched videos
            self.use_watched_checkbox.value = True
        self.use_watched_checkbox.observe(self.recluster, names="value")

        layout = Layout(width=f"{column_width}px", height="60px")
        self.choice_buttons = [
            Button(description="Expand", layout=layout)
            for _ in range(self.num_of_groups)
        ]
        for i, button in enumerate(self.choice_buttons):
            func = functools.partial(self.tree_climber.choose_column, i=i)
            button.on_click(func)

        # build UI
        if filter_categories:
            self.categories_selector = Categories(G)
            bottom.append(self.categories_selector.checkboxes_vbox)
            # bind click on categories to recluster
            event = Event(
                source=self.categories_selector.checkboxes_vbox,
                watched_events=["click"],
            )
            event.on_dom_event(self.recluster)
        top = [
            go_back_button,
            self.hide_watched_checkbox,
            self.use_watched_checkbox,
        ]
        layout = Layout(grid_gap=f"{self.grid_gap - 4}px")
        self.whole_output = VBox(
            [
                self.image_output,
                HBox(top),
                self.message_output,
                HBox(self.choice_buttons, layout=layout),
                self.video_wall,
            ]
        )

        self.recluster(None)

    def get_categories_filter(self):
        if self.filter_categories:
            return self.categories_selector.get_checked_categories_list()
        else:
            return "all"

    def recluster(self, _):
        nodes_to_cluster = select_nodes_to_cluster(
            self.G,
            use_watched=self.use_watched_checkbox.value,
            categories=self.get_categories_filter(),
        )
        self._nodes = nodes_to_cluster
        self.video_wall.clear_output()
        self.message_output.clear_output()

        tree, img = cluster_subgraph(nodes_to_cluster, self.G, self.clustering_balance)

        with self.image_output:
            self.image_output.clear_output()
            display(Image(value=img.getvalue()))
        video_ids = tree.pre_order()

        self.recommender.compute_node_ranks(video_ids)
        self.tree_climber.reset(tree)

    def display_video_grid(self, ids):
        ids = np.transpose(ids).flatten()

        css_style = """
            <style>
            body {margin: 40px;}
            .wrapper {
              display: grid;
              grid-template-columns:%s;
              grid-gap: %spx;
            }
            </style>
        """ % (
            f"{self.column_width}px " * self.num_of_groups,
            self.grid_gap,
        )
        html = '<div class="wrapper">'
        for id_ in ids:
            if id_ is None or G.nodes[id_].get("is_down"):
                # it's None if its cluster turned out empty after filtering
                # it can also be down
                # show an empty slot
                html += "<div></div>"
                continue

            video_url = id_to_url.format(id_)
            image_url = id_to_thumbnail.format(id_)
            title = self.G.nodes[id_]["title"]

            rank = self.recommender.node_ranks.get(id_)
            likes_to_views = liked_to_views_ratio(self.G, id_)
            likes_to_views = int(likes_to_views * 1000)
            info = f"rank: {rank}   l/v: {likes_to_views}"

            html += f"""<div>
                <a href="{video_url}"><img src="{image_url}" style='width: 100%; object-fit: contain'/></a>
                <a href="{video_url}" style="color:#EEEEEE; display: flex; flex-direction: column; height: 7em">
                    <span>{info}</span>
                    <span>{title}</span>
                </a>
            </div>"""
        html += "</div>"
        display(HTML(css_style + html))

    def update_displayed_videos(self, _):
        all_ids_to_show = []

        for grandchildren_from_a_child in self.tree_climber.grandchildren:
            ids_to_show = []
            for grandchild in grandchildren_from_a_child:
                # this line is the speed bottleneck
                ids = grandchild.pre_order()

                # filter ids
                if self.hide_watched_checkbox.value:
                    ids = only_not_watched(G, ids)

                id_to_show = self.recommender.recommend_by_in_degree(ids)
                ids_to_show.append(id_to_show)
            all_ids_to_show.append(ids_to_show)

        # scrape all the absent videos
        to_scrape = [id_ for sub in all_ids_to_show for id_ in sub]
        to_scrape = [id_ for id_ in to_scrape if id_ is not None]
        scrape_from_list(
            to_scrape, self.G, skip_if_fresher_than=float("inf"), non_verbose=True
        )

        # print("total videos: ", child.count)
        with self.video_wall:
            self.video_wall.clear_output(wait=True)
            self.display_video_grid(all_ids_to_show)

        save_graph(self.G)


parameters = dict(
    num_of_groups=3,
    videos_in_group=5,
    clustering_balance=1.4,
    recommendation_cutoff=0.9,
    column_width=260,
    seed=None,
)

# load query parameters
import os
from urllib.parse import parse_qs

query_string = os.environ.get("QUERY_STRING", "")
query_parameters = parse_qs(query_string)

for param_name in parameters.keys():
    if param_name in query_parameters:
        # parameters is a dict of lists
        parameters[param_name] = float(query_parameters[param_name][0])


ui = UI(G, **parameters)
ui.whole_output